# 03: Polygon Triangulation

*Authors: Jan Erik Swiadek, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases implementations of algorithms and data structures presented in the lecture, and it elaborates on some practical considerations concerning their use.
Furthermore, it provides interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Algorithms  
    3.1. Recursive Triangulation  
    3.2. Monotone Partitioning  
    3.3. Piecewise Triangulation  
4. Art Gallery Problem  
5. References  

## 1. Introduction

A **polygon triangulation** is ...

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external modules, including our module for generic data structures, our module for common geometric primitives and operations as well as our module for visualisation purposes. (The data structure and geometry modules will probably receive their own notebook later.)

In [ ]:
from typing import Optional

# Data structure, geometry and visualisation module imports
from modules.geometry import Point, LineSegment, DoublyConnectedPolygon, Vertex, HalfEdge
from modules.visualisation import VisualisationTool, SimplePolygonInstance, LineSegmentsMode

Additionally, we create an object for our visualisation tool and register a few example instances.

In [ ]:
visualisation_tool = VisualisationTool(400, 400, SimplePolygonInstance())   # TODO: Probably should change PolygonMode behaviour...

square = DoublyConnectedPolygon()
square.add_vertex(Point(150, 150))
square.add_vertex(Point(150, 250))
square.add_vertex(Point(250, 250))
square.add_vertex(Point(250, 150))
square.close()
visualisation_tool.register_example_instance("square", square)

## 3. Algorithms

...

### 3.1. Recursive Triangulation

Explanations ...

In [ ]:
# TODO: Input polygon should be simple, closed, have at least 3 vertices and no diagonals (maybe encode in type). Should the input be copied?
def recursive_pt(polygon: DoublyConnectedPolygon) -> DoublyConnectedPolygon:
    if polygon._number_of_vertices <= 3:
        return polygon
    anchor_edges = [polygon._root_vertex._edge]
    while anchor_edges:
        leftmost_edge = get_leftmost_edge(anchor_edges.pop())
        connection_edges = get_connection_edges(leftmost_edge)
        if connection_edges is not None:
            diagonal = polygon.add_diagonal(connection_edges[0], connection_edges[1])
            anchor_edges.extend((diagonal, diagonal._twin))
    return polygon

def get_leftmost_edge(anchor_edge: HalfEdge) -> HalfEdge:
    leftmost_edge, leftmost_vertex = anchor_edge, anchor_edge._origin
    edge = anchor_edge._next
    while edge is not anchor_edge:
        vertex = edge._origin
        if vertex.x < leftmost_vertex.x or (vertex.x == leftmost_vertex.x and vertex.y < leftmost_vertex.y):
            leftmost_edge, leftmost_vertex = edge, vertex
        edge = edge._next
    return leftmost_edge

def get_connection_edges(leftmost_edge: HalfEdge) -> Optional[tuple[HalfEdge, HalfEdge]]:
    edge = leftmost_edge._next._next
    if edge is leftmost_edge._prev:
        return None

    triangle = (leftmost_edge._prev._origin, leftmost_edge._origin, leftmost_edge._next._origin)
    connection_edge, max_coordinate = None, 0.0
    while edge is not leftmost_edge._prev:
        area_coordinates = get_area_coordinates(edge._origin, triangle)
        if all(0.0 <= coordinate <= 1.0 for coordinate in area_coordinates):
            if connection_edge is None or area_coordinates[1] > max_coordinate:
                connection_edge, max_coordinate = edge, area_coordinates[1]
        edge = edge._next
    if connection_edge is not None:
        return leftmost_edge, connection_edge

    return leftmost_edge._prev, leftmost_edge._next

def get_area_coordinates(vertex: Vertex, triangle: tuple[Vertex, Vertex, Vertex]) -> tuple[float, float, float]:
    parallelogram_area = signed_area(triangle[0], triangle[1], triangle[2])
    return (
        signed_area(vertex, triangle[1], triangle[2]) / parallelogram_area,
        signed_area(triangle[0], vertex, triangle[2]) / parallelogram_area,
        signed_area(triangle[0], triangle[1], vertex) / parallelogram_area
    )

def signed_area(u: Vertex, v: Vertex, w: Vertex) -> float:
    p, q, r = u._point, v._point, w._point
    return (q - p).cross(r - p)

In [ ]:
#recursive_pt(square)

Runs in $O(n^2)$.

Now visualisation.

In [ ]:
visualisation_tool.register_algorithm("rec", recursive_pt, LineSegmentsMode(endpoint_radius = 0.0))
# TODO: Implement appropriate mode. Maybe make Triangulation object.

If you haven't used our interactive visualisation tool before, see the convex hull notebook for an explanation. ...

In [ ]:
visualisation_tool.display()

### 3.2. Monotone Partitioning

...

### 3.3. Piecewise Triangulation

...

## 4. Art Gallery Problem

Maybe use the simple approach from [3] to solve the *Art Gallery Problem* ... the dual graph approach is explained in [1] but it's more involved.

## 5. References

[1] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd Edition, 2008.

[2] David M. Mount. [*CMSC 754: Computational Geometry*](https://www.cs.umd.edu/class/spring2020/cmsc754/Lects/cmsc754-spring2020-lects.pdf), 2020.

[3] Ali A. Kooshesh, and Bernard M. E. Moret. *Three-coloring the Vertices of a Triangulated Simple Polygon*, Pattern Recognition 25(4), p. 443, 1992.